<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example_5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 203 (delta 90), reused 73 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (203/203), 3.45 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/sp-pa-gep


In [ ]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Problem data (from the draft)
# -----------------------------
q = np.array([1, -2, -1, 2, -1], dtype=float)

P = np.array([
    [3.1, 2.0, 0,   0,   0],
    [2.0, 3.6, 0,   0,   0],
    [0,   0,   3.5, 2.0, 0],
    [0,   0,   2.0, 3.3, 0],
    [0,   0,   0,   0,   3.0],
], dtype=float)

Q = np.array([
    [1.6, 1.0, 0,   0,   0],
    [1.0, 1.6, 0,   0,   0],
    [0,   0,   1.5, 1.0, 0],
    [0,   0,   1.0, 1.5, 0],
    [0,   0,   0,   0,   2.0],
], dtype=float)

n = 5

# -----------------------------
# Sets K and C(x)
# -----------------------------
def proj_box(x, lo, hi):
    return np.minimum(np.maximum(x, lo), hi)

def C_upper(x):
    """Return the upper bound 'a(x)' so that C(x) = [0, a(x)]^5, with cap <= 2."""
    s = float(np.sum(x))
    if s <= 5.0:
        a = 1.0
    else:
        a = (s + 5.0) / 10.0
    # Ensure C(x) subset of K=[0,2]^5
    return min(a, 2.0)

def proj_C_of(x, y):
    """Project y onto C(x) = [0, a(x)]^5."""
    a = C_upper(x)
    return proj_box(y, 0.0, a)

def proj_K(y):
    return proj_box(y, 0.0, 2.0)

# -----------------------------
# Inner solver for resolvent (approx)
# -----------------------------
def solve_inner_qp_on_box(u_ref, x_n, r, y0=None, inner_it=300, inner_tol=1e-10):
    """
    Approximately solve a strongly convex quadratic over C(u_ref) using projected gradient:

        minimize_y  0.5 y^T (Q + (1/r) I) y + (P u_ref + q - (1/r) x_n)^T y
        subject to  y in C(u_ref) = [0, a(u_ref)]^5.

    This is a practical surrogate for the regularized EP step when C depends on u.
    """
    A = Q + (1.0 / r) * np.eye(n)
    b = P @ u_ref + q - (1.0 / r) * x_n

    # Lipschitz constant for grad: L = lambda_max(A)
    L = np.linalg.eigvalsh(A).max()
    step = 1.0 / L

    if y0 is None:
        y = proj_C_of(u_ref, x_n.copy())
    else:
        y = proj_C_of(u_ref, y0.copy())

    for _ in range(inner_it):
        grad = A @ y + b
        y_new = proj_C_of(u_ref, y - step * grad)
        if np.linalg.norm(y_new - y) <= inner_tol:
            y = y_new
            break
        y = y_new

    return y

def resolvent_state_dependent(x_n, r=1.0, fp_it=50, fp_tol=1e-10, inner_it=300):
    """
    Fixed-point loop for the state-dependent resolvent:
      u in C(u), and u approximates the regularized equilibrium step.

    We iterate:
      u^{j+1} = argmin over C(u^j) of a convex QP (solved approximately).
    """
    # Start in K
    u = proj_K(x_n.copy())

    for _ in range(fp_it):
        u_next = solve_inner_qp_on_box(u_ref=u, x_n=x_n, r=r, y0=u, inner_it=inner_it)
        u_next = proj_K(u_next)  # safety (should already lie in [0,2]^5)
        if np.linalg.norm(u_next - u) <= fp_tol:
            u = u_next
            break
        u = u_next

    return u

def residual_fn(x):
    u = resolvent_state_dependent(x, r=1.0, fp_it=60, inner_it=400)
    return float(np.linalg.norm(x - u))

# -----------------------------
# SPPA / WPPA outer iterations
# -----------------------------


In [ ]:
import src.benchkit as bk
import time

r = 1.0
maxit = 200
fp_it = 60
inner_it = 400
stop_tol = 0.0

def make_sppa_step():
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)
        u = resolvent_state_dependent(x, r=1.0, fp_it=60, inner_it=400)
        x_new = alpha * u_anchor + (1.0 - alpha) * u
        return x_new, {"u": u}   # <-- IMPORTANT
    return step_fn


def make_wppa_step():
    def step_fn(x, k):
        alpha = 1.0 / (k + 2.0)
        u = resolvent_state_dependent(x, r=1.0, fp_it=60, inner_it=400)
        x_new = alpha * x + (1.0 - alpha) * u
        return x_new, {"u": u}   # <-- IMPORTANT
    return step_fn

In [ ]:
x0 = np.ones(n) * 1.8  # any point in [0,2]^5
u_anchor = np.zeros(n) # Halpern anchor

logs_sppa, sum_sppa = bk.run(
    method_name="SPPA",
    x0=x0,
    max_iter=maxit,
    stop_tol_step=stop_tol,
    step_fn=make_sppa_step(),
    residual_fn=residual_fn,   # cheap placeholder
    error_fn=None,
)

logs_wppa, sum_wppa = bk.run(
    method_name="WPPA",
    x0=x0,
    max_iter=maxit,
    stop_tol_step=stop_tol,
    step_fn=make_wppa_step(),
    residual_fn=residual_fn,
    error_fn=None,
)


bk.make_standard_plots(
    logs_by_method={"SPPA": logs_sppa, "WPPA": logs_wppa},
    outdir="figures",
    tag="ex53_qep_gep",
    plot_residual=True,
    plot_error=False,
)

print(logs_sppa[-1].residual, logs_wppa[-1].residual)

print(bk.latex_table(
    [sum_sppa, sum_wppa],
    caption="CPU/accuracy summary for Example 5.3 (state-dependent GEP).",
    label="tab:ex53",
))

0.005973274881583385 9.523493105234593e-13
\begin{table}[!ht]
\centering
\begin{tabular}{lrrrrr}
\hline
Method & Iters & Total (s) & Avg resolvent (s) & Avg residual (s) & Final $R(x_n)$\\
\hline
SPPA & 200 & 3.7898 & 0.01893 & 0.00000 & 5.9733e-03 \\
WPPA & 200 & 3.0559 & 0.01526 & 0.00000 & 9.5235e-13 \\
\hline
\end{tabular}
\caption{CPU/accuracy summary for Example 5.3 (state-dependent GEP).}
\label{tab:ex53}
\end{table}

